In [1]:
! pip install openai langchain langchain_openai

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/39/1e/9dc3ccee95d0e16e54e353d3c355bb7cc506d56a2dbb0a07bc739cc48eac/openai-1.52.0-py3-none-any.whl.metadata
  Using cached openai-1.52.0-py3-none-any.whl.metadata (24 kB)
  Obtaining dependency information for langchain from https://files.pythonhosted.org/packages/53/b9/290c361a4976947ac6fad11af0a4c11db0b5d8357dc3447d28c1ecd9a1a3/langchain-0.3.4-py3-none-any.whl.metadata
  Using cached langchain-0.3.4-py3-none-any.whl.metadata (7.1 kB)
  Obtaining dependency information for langchain_openai from https://files.pythonhosted.org/packages/66/ea/dcc59d9b818a4d7f25d4d6b3018355a0e0243a351b1d4ef8b26ec107ee00/langchain_openai-0.2.3-py3-none-any.whl.metadata
  Using cached langchain_openai-0.2.3-py3-none-any.whl.metadata (2.6 kB)
  Obtaining dependency information for httpx<1,>=0.23.0 from https://files.pythonhosted.org/packages/5

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-cloud-auth 0.1.3 requires pydantic<2.0, but you have pydantic 2.9.2 which is incompatible.


In [2]:
import os
import pandas as pd
import pyarrow.parquet as pq
import re
import openai
import json
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document
#from langchain.schema import Document

In [3]:
os.environ['OPENAI_API_KEY'] = ''
openAIModelName = 'gpt-4-turbo'
# 3.5
llm_mod1 = ChatOpenAI(model_name = openAIModelName,
                        temperature=0.3)

In [8]:
template_forrajes = """Actúa como observador de colectas que encuentra y extrae la información solicitada del texto. Te voy a proporcionar un texto largo extraído de un PDF que es un informe de colecta.
Necesito que extraigas TODA la información específica del texto y de todas las tablas donde encuentres altitud, latitud y longitud y la estructures en formato JSON que contenga todas las accesiones 
relacionadas en esa colección y asignar a cada una la información correspondiente a los siguientes campos:CIAT No., pais, provincia/estado,fuente de germoplasma (si es 1 es colección CIAT, si es 2 es Coleccion conjunta, si es 3 es Donacion. En caso de que no se entienda, tomar esta información de la segunda fila: si es 1 es coleccion CIAT, si es 2 es Donacion, o si no encuentras esta información, tomarla de "Observaciones"), nombre del colector o donante, especie, fecha de colecta o Fecha Col./Intr (SOLO SI HAY información), fecha de registro, lugar de recolección (TODA la información que encuentres), altitud, longitud, latitud. 
Puede haber más información o menos información que la mencionada. El objetivo que no debes dejar pasar es el identificador de la semilla, altitud, longitud y latitud en TODO el documento y tablas. NO extraigas información de accesiones duplicadas, cada documento corresponde a UN SOLO NÚMERO DE accesión, por lo tanto el json debe tener una sola accesion. No extraigas información sobre la topografía o hábitat nativo, tampoco la palabra carretera si no hay algo escrito ahí. Si alguno de los campos solicitados no está disponible en el documento, por favor asigna un "no especificado", pero no te saltes algún campo.

Este es un ejemplo de como deberia resultar la data de una accesión:

    "CIAT No.": "1",
    "especie": "Stylosanthes scabra",
    "fuente de germoplasma": "Donacion",
    "colectores": "CSIRO Townsville, Australia",
    "Pais": "Brasil",
    "Provincia/Estado": "Sao Paulo",
    "Lugar recoleccion": "Pedro Branca, Sao Paulo",
    "fecha de colecta": "01/01/1968",
    "fecha de registro": "28/07/1971",
    "altitud": "No especificada",
    "longitud": "75.34W",
    "latitud": "02.57N"  
    
Aquí está el texto real del PDF que necesito que proceses:"""

human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template_forrajes),
    ("human", human_template),
])

# No. Donante (si la fuente de germoplasma es 3 o si en la segunda fila hay un 2), No. Colector (si la fuente de germoplasma es 1), No. Campo (si la fuente de germoplasma es 2),

# Funcion para extraer datos de pdf

In [ ]:
def filter_filenames_by_length(directory_path, pattern):
    matching_files = []
    for filename in [os.path.join(directory, file) for file in os.listdir(directory)]:
        if re.match(pattern, filename) and 'combined_data' not in filename :
            matching_files.append(filename)
    return matching_files

directory = 'Path to the extractions files' #'/home/ec2-user/SageMaker/pdf_extractions/data/3_out_data/forrajes'
parquet_file_path = [os.path.join(directory, file) for file in os.listdir(directory)]
files = filter_filenames_by_length(directory, r".*\.parquet$")

In [ ]:
def extract_number(file_path):
    # Function to extract the numeric part from the file path

    match = re.search(r'/(\d+)\.parquet$', file_path)
    return int(match.group(1)) if match else float('inf')
 
 
# Sort the list of file paths based on the numeric part
sorted_file_paths = sorted(files, key=extract_number)

In [ ]:
def extract_pdf_info(parquet_file, openAIModelName, temperature, chat_prompt):

    # Function to extract PDF data

   
    print(f"Processing the file {parquet_file}")
   
    llm_mod1 = ChatOpenAI(model_name = openAIModelName,
                        temperature=temperature)
    df = pd.read_parquet(parquet_file, engine='pyarrow')
    text = '\n'.join(df['Texto_Extraido'].tolist())
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    docs = [Document(page_content=x) for x in text_splitter.split_text(text)]
   
    try:
        messages = chat_prompt.format_messages(text=docs)
        result = llm_mod1.invoke(messages)
        output = result.content
       
        print(output)
       
        # Extract JSON data
        start_index = output.find('{')
        end_index = output.rfind('}') + 1
        json_string = output[start_index:end_index]
        json_data = json.loads(json_string)
       
        return json_data
       
    except :
        print("Failed to decode JSON from the result content.")

In [ ]:
#Run the function for all the parquet files

results = list()
for file in sorted_file_paths:
    results.append(extract_pdf_info(file, openAIModelName = openAIModelName, temperature = 0.3, chat_prompt = template_forrajes))

In [ ]:
def append_json_data(file,output_path):
    
    # append all the results in a txt
   
        json_dict = {}
        for json_data in file:
                first_key = list(json_data.keys())[0]
                first_value = json_data[first_key]
                json_dict[first_value] = json_data
 
        # Convert the dictionary to a JSON-formatted string
        json_string = json.dumps(json_dict, indent=4)
 
        # Save the JSON string to a text file
        with open(output_path, 'w', encoding='utf-8') as file:
            file.write(json_string)
       
       
        print(f"Output appended to {output_path}")
       
        return json_dict

In [ ]:
#Append all the outputs and save them

output_folder = 'Path to write the output'

dicts = append_json_data(results,  os.path.join(output_folder,'output_extractions.txt'))
  
with open('data/output_extractions.txt', 'rb') as file:
        dictio = json.load(file)
 

In [ ]:
#Convert txt to df

df = pd.DataFrame(dictio)
df_reset = df.reset_index()
df_reset = df_reset.transpose()

In [ ]:
#Save final dataframe

df_reset.to_csv(os.path.join(output_folder,'output_extractions.csv'), header = False, index = False, encoding ='utf-8')

## Run try one by one
#### DONT RUN

In [ ]:
#parquet_file_path = '/home/ec2-user/SageMaker/genebanks/data/3_out_data/Ecogeographic survey of the target species-Costa Rica-2004.parquet'
#df = pd.read_parquet(parquet_file_path, engine='pyarrow')

In [5]:
df_forrajes = pd.read_csv("C:/Users/macosta/Downloads/combined_data_forrajes.csv")
df_forrajes = df_forrajes[df_forrajes['Nombre_Carpeta'] == 1838]
df_forrajes.head()

,Nombre_Carpeta,Nombre_Imagen,Texto_Extraido,conteo_caracteres,Numero_Pagina
102,1838,page_1.png,1838\nCALA\nPrograma de Pastos Tropicales\nCIA...,1968,1
103,1838,page_2.png,TARJETA DE RECOLECCION /INTRODUCCION\nCIAT No....,896,2


In [10]:
# Concatenar el texto extraído de todas las páginas
text1 = '\n'.join(df_forrajes['Texto_Extraido'].tolist())
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs1 = [Document(page_content=x) for x in text_splitter.split_text(text1)]

Created a chunk of size 1990, which is longer than the specified 1000


In [11]:
messages1 = chat_prompt.format_messages(text=docs1)
result1 = llm_mod1.invoke(messages1)
output1 = result1.content
print(result1.content)

```json
{
    "CIAT No.": "1838",
    "especie": "Stylosanthes guianensis",
    "fuente de germoplasma": "Colección conjunta",
    "colectores": "R. Schultze-Kraft, C. Ortega, B. Castillo",
    "Pais": "Panamá",
    "Provincia/Estado": "Coclé",
    "Lugar recoleccion": "Penonomé-Toabré, 4 km al Nordeste de La Pintada",
    "fecha de colecta": "24/01/1978",
    "fecha de registro": "31/01/1978",
    "altitud": "100 m.s.n.m.",
    "longitud": "80W25",
    "latitud": "8N38"
}
```
